In [1]:
import numpy as np
import cv2 as cv
import glob
%matplotlib inline 
from matplotlib import pyplot as plt

In [2]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)

patternx = 10;
patterny = 7
objp = np.zeros((patternx*patterny,3), np.float32)
objp[:,:2] = np.mgrid[0:patternx,0:patterny].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('*.bmp')

In [3]:
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (10,7), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        #cv.drawChessboardCorners(img, (10,7), corners2, ret)
        #plt.imshow(img)
        #cv.waitKey(500)
    if ret == False:
        images.remove(fname)


## Calculate Distortion Properties

In [7]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Test Single Image Calibration


In [4]:
#input_image = cv.imread('r_00003.bmp')

#rgb_image=cv.cvtColor(input_image, cv.COLOR_BGR2RGB)
#gray = cv.cvtColor(input_image, cv.COLOR_BGR2GRAY)
#plt.imshow(gray, cmap='gray')

In [5]:
#ret, corners = cv.findChessboardCorners(gray, (10,7), None)

In [6]:
#corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
# Draw and display the corners
#cv.drawChessboardCorners(gray, (10,7), corners2, ret)
#plt.imshow(gray, cmap='gray')

        

# Undistort

In [8]:
#img = cv.imread('r_00003.bmp')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [9]:
# undistort
#dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
#x, y, w, h = roi
#dst = dst[y:y+h, x:x+w]
#cv.imwrite('calib/r_00003_calib.png', dst)

## Bulk Undistort

In [14]:
images = glob.glob('*.bmp')
for fname in images:
    img = cv.imread(fname)
    # undistort
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    cv.imwrite('calib/' + fname.replace('.bmp','_calib.bmp'), dst)    
    #print(fname.replace('.bmp','_calib.bmp'))

# Re-projection Error

Re-projection error gives a good estimation of just how exact the found parameters are. The closer the re-projection error is to zero, the more accurate the parameters we found are. Given the intrinsic, distortion, rotation and translation matrices, we must first transform the object point to image point using cv.projectPoints(). Then, we can calculate the absolute norm between what we got with our transformation and the corner finding algorithm. To find the average error, we calculate the arithmetical mean of the errors calculated for all the calibration images.


In [12]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.017142123795271337
